<a href="https://colab.research.google.com/github/F-Emm/Microsoft-Capstone-Project/blob/main/Movie_Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Dependencies

In [ ]:
!pip install requests fuzzywuzzy pandas pyjstat numpy plotly matplotlib seaborn geopy google folium pandas googlemaps

In [31]:
import pandas as pd
# import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import process
import pickle
import joblib

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')

###This is a kaggle dataset from [link](https://www.kaggle.com/code/danielpry/netflix-recommendation-system) below...

https://www.kaggle.com/code/danielpry/netflix-recommendation-system

In [1]:
#loading this from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movie_titles = pd.read_csv('/content/drive/MyDrive/csv files/Kaggle Datasets/Netflix TV Shows/titles.csv')

In [5]:
# Checking data type of each column
movie_titles.dtypes

id                       object
title                    object
type                     object
description              object
release_year              int64
age_certification        object
runtime                   int64
genres                   object
production_countries     object
seasons                 float64
imdb_id                  object
imdb_score              float64
imdb_votes              float64
tmdb_popularity         float64
tmdb_score              float64
dtype: object

##Cleaning Data

####*In this section we will prepare the dataset to get that raw data which can be used with K Nearest Neighbors in order to make predictions. K Nearest Neighbors is a good model for this project since its main assumption is that data points that are close to each other are alike. Which should hold as many of the attributes such as genre, release year, IMDb score, and whether or not it is a movie or tv show should ensure that similar titles are close to one another and allow recommendations to be made by selecting titles closest to that specific title or genre type.*
Also clearing whitespaces


In [14]:
# looking what columns contain null values
amount_missing = movie_titles.isnull().sum().sort_values(ascending = False) / len(movie_titles) * 100
missing_values = pd.DataFrame({'Features': amount_missing.index, 'Null_Value_Percentage': amount_missing.values}).style.background_gradient(cmap='Accent_r')
missing_values

,Features,Null_Value_Percentage
0,seasons,64.743369
1,age_certification,44.953496
2,imdb_votes,9.283500
3,imdb_score,9.007923
4,imdb_id,7.647261
5,tmdb_score,5.477093
6,tmdb_popularity,1.619015
7,description,0.310024
8,title,0.017224
9,id,0.000000


In [19]:
# Finding all the unique genres
array = movie_titles.genres.to_list()

array1 = [s.strip("[]") for s in array]

lst = []
for i in range(len(array1)):
    lst.append(array1[i].split(','))

result = list({x for l in lst for x in l})

result1 = [s.strip("' ") for s in result]

# function to get unique values
def unique(list1):
      
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    for x in unique_list:
        print(x),

results = unique(result1)


european
documentation
action
drama
crime
romance
fantasy
music
sport
family
history
animation
thriller
western
scifi
war
horror
reality
comedy


'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_...

#####In the block above we clean the genres column and output all of the unique genres that show up in the genres column. With this we engineer new columns that contain 1 (True) and 0 (False) if that specific genre is listed in the genres column.

In [20]:
# Creating indictor columns for each genre
movie_titles['horror'] = movie_titles['genres'].str.contains('horror').astype('int')
movie_titles['history'] = movie_titles['genres'].str.contains('history').astype('int')
movie_titles['european'] = movie_titles['genres'].str.contains('european').astype('int')
movie_titles['sport'] = movie_titles['genres'].str.contains('sport').astype('int')
movie_titles['family'] = movie_titles['genres'].str.contains('family').astype('int')
movie_titles['reality'] = movie_titles['genres'].str.contains('reality').astype('int')
movie_titles['drama'] = movie_titles['genres'].str.contains('drama').astype('int')
movie_titles['animation'] = movie_titles['genres'].str.contains('animation').astype('int')
movie_titles['comedy'] = movie_titles['genres'].str.contains('comedy').astype('int')
movie_titles['scifi'] = movie_titles['genres'].str.contains('scifi').astype('int')
movie_titles['music'] = movie_titles['genres'].str.contains('music').astype('int')
movie_titles['fantasy'] = movie_titles['genres'].str.contains('fantasy').astype('int')
movie_titles['western'] = movie_titles['genres'].str.contains('western').astype('int')
movie_titles['thriller'] = movie_titles['genres'].str.contains('thriller').astype('int')
movie_titles['documentation'] = movie_titles['genres'].str.contains('documentation').astype('int')
movie_titles['crime'] = movie_titles['genres'].str.contains('crime').astype('int')
movie_titles['action'] = movie_titles['genres'].str.contains('action').astype('int')
movie_titles['war'] = movie_titles['genres'].str.contains('war').astype('int')
movie_titles['romance'] =movie_titles['genres'].str.contains('romance').astype('int')
movie_titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,...,scifi,music,fantasy,western,thriller,documentation,crime,action,war,romance
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,48,['documentation'],['US'],1.0,...,0,0,0,0,0,1,0,0,0,0
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,...,0,0,0,0,0,0,1,0,0,0
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,...,0,0,1,0,0,0,0,0,0,0
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,...,0,0,0,0,0,0,0,0,0,0
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,...,0,0,0,0,0,0,0,0,0,0
5,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,...,0,0,0,0,0,0,0,0,0,0
6,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'crime', 'action']",['US'],NaN,...,0,0,0,0,1,0,1,1,0,0
7,tm185072,My Fair Lady,MOVIE,A snobbish phonetics professor agrees to a wag...,1964,G,170,"['drama', 'music', 'romance', 'family']",['US'],NaN,...,0,1,0,0,0,0,0,0,0,1
8,tm98978,The Blue Lagoon,MOVIE,Two small children and a ship's cook survive a...,1980,R,104,"['romance', 'drama']",['US'],NaN,...,0,0,0,0,0,0,0,0,0,1
9,tm119281,Bonnie and Clyde,MOVIE,"In the 1930s, bored waitress Bonnie Parker fal...",1967,R,110,"['drama', 'crime', 'action']",['US'],NaN,...,0,0,0,0,0,0,1,1,0,0


#####Here unneeded columns are dropped as it might not help our prediction in anyway(either because it's common through out or it doesn't classify the movie type i.e: rating). Also show and movie columns with 1 and 0 denoting True and False are created from the type column.

In [21]:
# dropping unneeded columns
movie_titles_modify = movie_titles.drop(columns = ['title', 'description', 'genres', 'imdb_id', 'production_countries'])
movie_titles_modify

# creating indicator columns for show or movie
movie_titles_modify['movie'] = movie_titles_modify['type'].str.contains('MOVIE').astype('int')
movie_titles_modify['show'] = movie_titles_modify['type'].str.contains('SHOW').astype('int')
movie_titles_modify = movie_titles_modify.drop(columns = 'type')

#####In the part we make more indicator columnsfor each unique age certification level. Since there are missing values in the age certification column... this missing values are denoted with 1 in the not rated indicator column, showing that yes this Movie was not rated.

In [22]:
# creating indicator columns for age certification
unique(movie_titles_modify['age_certification'])

movie_titles_modify['not_rated'] = movie_titles_modify['age_certification'].isna().astype('int')
movie_titles_modify['TV-Y'] = movie_titles_modify['age_certification'].str.contains('TV-Y', na = False).astype('int')
movie_titles_modify['TV-PG'] = movie_titles_modify['age_certification'].str.contains('TV-PG', na = False).astype('int')
movie_titles_modify['PG'] = movie_titles_modify['age_certification'].str.contains('PG', na = False).astype('int')
movie_titles_modify['R'] = movie_titles_modify['age_certification'].str.contains('R', na = False).astype('int')
movie_titles_modify['TV-14'] = movie_titles_modify['age_certification'].str.contains('TV-14', na = False).astype('int')
movie_titles_modify['PG-13'] = movie_titles_modify['age_certification'].str.contains('PG-13', na = False).astype('int')
movie_titles_modify['TV-Y7'] = movie_titles_modify['age_certification'].str.contains('TV-Y7', na = False).astype('int')
movie_titles_modify['NC-17'] = movie_titles_modify['age_certification'].str.contains('NC-17', na = False).astype('int')
movie_titles_modify['G'] = movie_titles_modify['age_certification'].str.contains('G', na = False).astype('int')
movie_titles_modify['TV-G'] = movie_titles_modify['age_certification'].str.contains('TV-G', na = False).astype('int')
movie_titles_modify['TV-MA'] = movie_titles_modify['age_certification'].str.contains('TV-MA', na = False).astype('int')
movie_titles_modify = movie_titles_modify.drop(columns = ['age_certification'])
movie_titles_modify.head()

nan
TV-PG
TV-14
G
PG-13
TV-G
TV-Y
R
TV-Y7
NC-17
TV-MA
PG


,id,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,horror,history,...,TV-PG,PG,R,TV-14,PG-13,TV-Y7,NC-17,G,TV-G,TV-MA
0,ts300399,1945,48,1.0,NaN,NaN,0.600,NaN,0,0,...,0,0,0,0,0,0,0,0,0,1
1,tm84618,1976,113,NaN,8.3,795222.0,27.612,8.2,0,0,...,0,0,1,0,0,0,0,0,0,0
2,tm127384,1975,91,NaN,8.2,530877.0,18.216,7.8,0,0,...,0,1,0,0,0,0,0,1,0,0
3,tm70993,1979,94,NaN,8.0,392419.0,17.505,7.8,0,0,...,0,0,1,0,0,0,0,0,0,0
4,tm190788,1973,133,NaN,8.1,391942.0,95.337,7.7,1,0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
movie_titles_modify.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,horror,history,european,...,TV-PG,PG,R,TV-14,PG-13,TV-Y7,NC-17,G,TV-G,TV-MA
count,5806.000000,5806.000000,2047.000000,5283.000000,5.267000e+03,5712.000000,5488.000000,5806.000000,5806.000000,5806.000000,...,5806.000000,5806.000000,5806.000000,5806.000000,5806.000000,5806.000000,5806.000000,5806.000000,5806.00000,5806.00000
mean,2016.013434,77.643989,2.165608,6.533447,2.340719e+04,22.525660,6.818039,0.065450,0.040131,0.079228,...,0.032036,0.150189,0.099035,0.080951,0.075784,0.019290,0.002411,0.185842,0.01309,0.14485
std,7.324883,39.474160,2.636207,1.160932,8.713432e+04,68.849177,1.171560,0.247339,0.196283,0.270118,...,0.176110,0.357288,0.298735,0.272783,0.264674,0.137556,0.049050,0.389013,0.11367,0.35198
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
25%,2015.000000,44.000000,1.000000,5.800000,5.210000e+02,3.155250,6.100000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
50%,2018.000000,84.000000,1.000000,6.600000,2.279000e+03,7.478000,6.900000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
75%,2020.000000,105.000000,2.000000,7.400000,1.014400e+04,17.775750,7.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
max,2022.000000,251.000000,42.000000,9.600000,2.268288e+06,1823.374000,10.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000


#####In the following block missing values are filled with 0 and all the columns are normalized from 0 to 1. This normalization is done to ensure that none of the columns have a greater weight when passed into the K Nearest Neighbor model that will be used for making recommendation's.

In [24]:
# Changing NA's to zeros
movie_titles_modify = movie_titles_modify.fillna(0).drop(columns='id')
movie_titles_modify.head()

# Normalizing data
normalized_data = (movie_titles_modify - movie_titles_modify.min())/(movie_titles_modify.max() - movie_titles_modify.min())
normalized_data.head()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,horror,history,european,...,TV-PG,PG,R,TV-14,PG-13,TV-Y7,NC-17,G,TV-G,TV-MA
0,0.000000,0.191235,0.02381,0.000000,0.000000,0.000329,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.402597,0.450199,0.00000,0.864583,0.350582,0.015143,0.82,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.389610,0.362550,0.00000,0.854167,0.234043,0.009990,0.78,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.441558,0.374502,0.00000,0.833333,0.173002,0.009600,0.78,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.363636,0.529880,0.00000,0.843750,0.172792,0.052286,0.77,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##Making Model

#####We fit the K Nearest Neighbors model below and use cosine similarity as the metric to determine how close titles are from each other.

In [25]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
knn.fit(normalized_data)

NearestNeighbors(algorithm='brute', metric='cosine')

In [26]:
# creating df of title names
title_names = movie_titles.loc[ : , ['title']]
title_names['title'] = title_names['title'].str.lower()
title_names.head()

,title
0,five came back: the reference films
1,taxi driver
2,monty python and the holy grail
3,life of brian
4,the exorcist


In [46]:
#get the ten nearest neighbors
def recommendations(a):
    neighbor_index = knn.kneighbors([normalized_data.loc[a]], return_distance=False, n_neighbors=11)
    neighbor_index = list(neighbor_index[0])
    neighbor_index = neighbor_index[1:11]
    print("Recommendations for: " + title_names.loc[a, 'title'])
    print(title_names.loc[neighbor_index])

In [47]:
netflix_titles = list(title_names.loc[:, 'title'])

def movie(title):
    title = title.lower()
    result = process.extract(title, netflix_titles, limit = 5)
    i = 1
    for mov in result:
        if (i == 1 and mov[1] <= 80):
            print('No close matches found. Try again.')
            break
        if mov[1] == 100:
            index = netflix_titles.index(title)
            recommendations(index)
            return
        if (mov[1] >= 80 and mov[1] != 100):
            if i == 1:
                print('Exact match not found for {}. Did you mean:'.format(title))
            print(str(i) + ') ' + mov[0])
            i = i+1


#####In the following blocks some examples are shown using the function to show recommendations.


In [48]:
movie('breaking bad')

Recommendations for: breaking bad
                 title
1260            narcos
1824             ozark
1835        mindhunter
1826        the sinner
1840             elite
951          wentworth
2198     the mechanism
3362        undercover
4935  the chestnut man
1888       alias grace


In [49]:
movie('walking dead')
movie('the walking dead')

Exact match not found for walking dead. Did you mean:
1) the walking dead
2) adú
3) the quick and the dead
4) dead set
5) chhota bheem & krishna: pataliputra- city of the dead
Recommendations for: the walking dead
                           title
3241                black summer
4887          all of us are dead
3239                     kingdom
1850              altered carbon
1856  the haunting of hill house
1315                          3%
4909              the silent sea
1305                      glitch
4860                  archive 81
3272   the haunting of bly manor


In [ ]:
movie('too hot to handle')

In [ ]:
movie('star wars')
movie('star trek: deep space nine')

##Saving the model

In [40]:
movie_model = knn

In [41]:
movie_modelfile = open('movie_model_file', 'wb')

In [43]:
pickle.dump(movie_model, movie_modelfile)

In [50]:
# load the model from disk
loaded_model = pickle.load(open('movie_model_file', 'rb'))
# result

def recommended(a):
    neighbor_index = loaded_model.kneighbors([normalized_data.loc[a]], return_distance=False, n_neighbors=11)
    neighbor_index = list(neighbor_index[0])
    neighbor_index = neighbor_index[1:11]
    print("Recommendations for: " + title_names.loc[a, 'title'])
    print(title_names.loc[neighbor_index])

In [ ]:
netflix_titles = list(title_names.loc[:, 'title'])

def movie(title):
    title = title.lower()
    result = process.extract(title, netflix_titles, limit = 5)
    i = 1
    for mov in result:
        if (i == 1 and mov[1] <= 80):
            print('No close matches found. Try again.')
            break
        if mov[1] == 100:
            index = netflix_titles.index(title)
            recommended(index)
            return
        if (mov[1] >= 80 and mov[1] != 100):
            if i == 1:
                print('Exact match not found for {}. Did you mean:'.format(title))
            print(str(i) + ') ' + mov[0])
            i = i+1

####Let's Pickle the necessary datasets

In [52]:
normalized_data.head()
normal_datafile = open('normalized_data_file', 'wb')
pickle.dump(normalized_data, normal_datafile)

In [54]:
title_names.head()
title_namesfile = open('title_names_file', 'wb')
pickle.dump(title_names, title_namesfile)

In [ ]:
# load the data from disk
loaded_norm_data = pickle.load(open('normalized_data_file', 'rb'))

loaded_title_data = pickle.load(open('title_names_file', 'rb'))

In [ ]:
# joblib.dump(movie_model, '/content/drive/MyDrive/Colab Notebooks/Vessel Project/ml-web-app/model-deploy/model_bi.pkl')

#With this we can use the saved pickle(s) to create a ***Flask Web App...***

##End